In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import skimage as ski
import scipy.sparse
import time

In [ ]:
def forward_differences(M,N):
    row = np.arange(0,M*N)
    dat = np.ones(M*N)
    col = np.arange(0,M*N).reshape(M,N)
    col_xp = np.hstack([col[:,1:], col[:,-1:]])
    col_yp = np.vstack([col[1:,:], col[-1:,:]])
    
    FD1 = scipy.sparse.coo_matrix((dat, (row, col_xp.flatten())), shape=(M*N, M*N))- \
          scipy.sparse.coo_matrix((dat, (row, col.flatten())), shape=(M*N, M*N))

    FD2 = scipy.sparse.coo_matrix((dat, (row, col_yp.flatten())), shape=(M*N, M*N))- \
          scipy.sparse.coo_matrix((dat, (row, col.flatten())), shape=(M*N, M*N))
    
    FD = scipy.sparse.vstack([FD1, FD2])
    
    return FD

In [ ]:
def proj_inf_l2(p, tau):
    # size must be (K,N), l2 over K, inf over N
    norm_p = np.sqrt(np.sum(p**2, axis=0, keepdims=True))
    p /= np.maximum(1, norm_p/tau)
    return p

def proj_inf(p, tau):
    return np.clip(p, -tau, tau)

In [ ]:
# proximal gardient method for dual ROF model    
def rof_dual_pg(d, lamb=1.0, maxit=1000, check=100, verbose=0):
    
    M,N = d.shape
    d = d.reshape(M*N)
    u = np.zeros(M*N)

    # dual variable
    p = np.zeros(M*N*2)

    # make nabla operator
    D = forward_differences(M,N)

    # Lipschitz constant
    L = 8
    tau = 1/L
    t0 = time.time()
    G = []
    for it in range(0,maxit):

        # gradient step
        p = p - tau*(D@(D.T@p-d))
    
        # projection
        p = p.reshape(2,M*N)
        p = proj_inf_l2(p,lamb)
        p = p.reshape(2*M*N)
    
        # recover primal solution
        u = d - D.T@p
        
        TV1 = lamb*np.sum(np.sqrt(np.sum(((D@u).reshape(2,M*N))**2, axis=0)))
        TV2 = np.sum((D@u)*p)
        gap = TV1 - TV2 + 0.5*np.sum((d-D.T@p-u)**2)
        
        G.append(gap)
        if verbose > 0:
            if it%check == check-1:
                print("iter = ", it,
                      ", tau = ", "{:.3f}".format(tau),
                      ", time = ", "{:.3f}".format(time.time()-t0),
                      ", gap = ", "{:.3f}".format(gap),
                      end="\r")
                
    return u.reshape(M,N), p.reshape(2,M,N), np.array(G)

In [ ]:
# Load image
g = ski.io.imread("watercastle.png")/255.0
M,N = g.shape

# Add noise to the image
f = g + np.random.randn(M,N)*0.1

In [ ]:
# Solve the dual ROF model using proximal gradient
lamb_tv = 0.1
u, p, gap = rof_dual_pg(f, maxit=1000,lamb=lamb_tv, verbose=1)

In [ ]:
plt.figure()

plt.subplot(121)
plt.imshow(f, cmap="gray")

plt.subplot(122)
plt.imshow(u, cmap="gray")

In [ ]:
plt.figure()
rate=8*np.sum(p**2)/2/np.arange(1,len(gap),1)
plt.loglog(gap, linewidth=2.0, label="proximal gradient")
plt.loglog(rate, "k--", linewidth=2.0, label=r"$O(1/k)$")
plt.legend()

In [ ]:
# accelerated proximal gardient method for dual ROF model    
def rof_dual_apg(d, lamb=1.0, maxit=1000, check=100, verbose=0):
    
    M,N = d.shape
    d = d.reshape(M*N)
    u = np.zeros(M*N)

    # dual variable
    p = np.zeros(M*N*2)
    p_old = p.copy()

    # make nabla operator
    D = forward_differences(M,N)

    # Lipschitz constant
    L = 8
    tau = 1/L
    t0 = time.time()
    G = []
    tk=1.0
    for it in range(0,maxit):
        
        # compute optimal overrelaxationnparameter
        tkp=(1.0+np.sqrt(1.0+4.0*tk**2))/2.0
        beta = (tk-1.0)/tkp
        tk = tkp
        
        # compute extrapolated variable
        p_ = p + beta*(p-p_old)

        # remeber old
        p_old = p.copy()
        
        # gradient step
        p = p_ - tau*(D@(D.T@p_-d))
    
        # projection
        p = p.reshape(2,M*N)
        p = proj_inf_l2(p,lamb)
        p = p.reshape(2*M*N)
    
        # recover primal solution
        u = d - D.T@p
        
        TV1 = lamb*np.sum(np.sqrt(np.sum(((D@u).reshape(2,M*N))**2, axis=0)))
        TV2 = np.sum((D@u)*p)
        gap = TV1 - TV2 + 0.5*np.sum((d-D.T@p-u)**2)
        
        G.append(gap)
        if verbose > 0:
            if it%check == check-1:
                print("iter = ", it,
                      ", tau = ", "{:.3f}".format(tau),
                      ", time = ", "{:.3f}".format(time.time()-t0),
                      ", gap = ", "{:.6f}".format(gap),
                      end="\r")
                
    return u.reshape(M,N), p.reshape(2,M,N), np.array(G)

In [ ]:
# Solve the dual ROF model using accelerated proximal gradient
lamb_tv = 0.1
u, p, gap2 = rof_dual_apg(f, maxit=1000,lamb=lamb_tv, verbose=1)

In [ ]:
rate2=2*8*np.sum(p**2)/(1+np.arange(1,len(gap),1))**2
plt.loglog(gap2, linewidth=2.0, label="accelerated proximal gradient")
plt.loglog(rate2,"k-.", linewidth=2.0, label=r"$O(1/k^2)$")
plt.legend()